# 🚀 Vesuvius Challenge - Runpods環境用学習コード

Runpods環境で動作する完全な学習パイプライン

In [ ]:
# 環境セットアップ
import os
import sys
from pathlib import Path

# Runpods環境確認
print("🔍 環境確認中...")
print(f"現在のディレクトリ: {os.getcwd()}")
print(f"GPU利用可能: {torch.cuda.is_available() if 'torch' in sys.modules else 'PyTorchまだインポートされていません'}")

# データパス候補
data_candidates = [
    "/workspace/vesuvius-challenge-surface-detection",
    "/content/vesuvius-challenge-surface-detection",
    "./vesuvius-challenge-surface-detection",
    "../input/vesuvius-challenge-surface-detection"
]

print("\n📁 データ検索中...")
for path in data_candidates:
    if Path(path).exists():
        print(f"✅ データ発見: {path}")
        train_images = Path(path) / "train_images"
        if train_images.exists():
            tiff_count = len(list(train_images.glob("*.tif*")))
            print(f"   画像数: {tiff_count}")
    else:
        print(f"❌ 見つかりません: {path}")

In [ ]:
# 必要なライブラリインポート
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Optional, Tuple, List

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
# パスを追加
sys.path.append('../')

# 統合データローダーインポート
from src.unified_data_loader import create_data_loaders

print("✅ データローダーインポート完了")

## 📊 データローダー作成（Runpods自動検出）

In [ ]:
# データローダー作成（パス指定なし = 自動検出）
train_loader, val_loader = create_data_loaders(
    volume_size=(96, 96, 64),
    batch_size=4,
    train_samples=24,
    val_samples=6,
    data_path=None,  # Noneで自動検出
    num_workers=2
)

## 🏗️ 3D U-Netモデル定義

In [ ]:
class UNet3D(nn.Module):
    """3D U-Net for Vesuvius surface detection"""
    
    def __init__(self, in_channels=1, out_channels=2, init_features=32):
        super(UNet3D, self).__init__()
        
        features = init_features
        
        # Encoder
        self.encoder1 = self._block(in_channels, features, "enc1")
        self.pool1 = nn.MaxPool3d(kernel_size=2, stride=2)
        
        self.encoder2 = self._block(features, features * 2, "enc2")
        self.pool2 = nn.MaxPool3d(kernel_size=2, stride=2)
        
        self.encoder3 = self._block(features * 2, features * 4, "enc3")
        self.pool3 = nn.MaxPool3d(kernel_size=2, stride=2)
        
        self.encoder4 = self._block(features * 4, features * 8, "enc4")
        self.pool4 = nn.MaxPool3d(kernel_size=2, stride=2)
        
        # Bottleneck
        self.bottleneck = self._block(features * 8, features * 16, "bottleneck")
        
        # Decoder
        self.upconv4 = nn.ConvTranspose3d(features * 16, features * 8, kernel_size=2, stride=2)
        self.decoder4 = self._block((features * 8) * 2, features * 8, "dec4")
        
        self.upconv3 = nn.ConvTranspose3d(features * 8, features * 4, kernel_size=2, stride=2)
        self.decoder3 = self._block((features * 4) * 2, features * 4, "dec3")
        
        self.upconv2 = nn.ConvTranspose3d(features * 4, features * 2, kernel_size=2, stride=2)
        self.decoder2 = self._block((features * 2) * 2, features * 2, "dec2")
        
        self.upconv1 = nn.ConvTranspose3d(features * 2, features, kernel_size=2, stride=2)
        self.decoder1 = self._block(features * 2, features, "dec1")
        
        # Final convolution
        self.conv = nn.Conv3d(features, out_channels, kernel_size=1)
        
    def _block(self, in_channels, features, name):
        return nn.Sequential(
            nn.Conv3d(in_channels, features, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm3d(features),
            nn.ReLU(inplace=True),
            nn.Conv3d(features, features, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm3d(features),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))
        enc3 = self.encoder3(self.pool2(enc2))
        enc4 = self.encoder4(self.pool3(enc3))
        
        bottleneck = self.bottleneck(self.pool4(enc4))
        
        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.decoder4(dec4)
        
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(dec3)
        
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)
        
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)
        
        return self.conv(dec1)

print("✅ モデル定義完了")

## ⚙️ 学習設定

In [ ]:
# デバイス設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用デバイス: {device}")

# モデル初期化
model = UNet3D(in_channels=1, out_channels=2).to(device)

# 損失関数とオプティマイザー
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5)

# パラメータ数確認
total_params = sum(p.numel() for p in model.parameters())
print(f"モデルパラメータ数: {total_params:,}")

## 🎯 学習ループ

In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    
    for batch in tqdm(loader, desc="Training"):
        data = batch['data'].to(device)
        target = batch['target'].to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(loader)

def validate_epoch(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="Validation"):
            data = batch['data'].to(device)
            target = batch['target'].to(device)
            
            output = model(data)
            loss = criterion(output, target)
            
            total_loss += loss.item()
    
    return total_loss / len(loader)

print("✅ 学習関数定義完了")

In [ ]:
# 学習実行
num_epochs = 10
train_losses = []
val_losses = []
best_val_loss = float('inf')

print(f"\n🎓 学習開始 ({num_epochs}エポック)\n")

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    print("-" * 50)
    
    # 学習
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    
    # 検証
    val_loss = validate_epoch(model, val_loader, criterion, device)
    val_losses.append(val_loss)
    
    # スケジューラー更新
    scheduler.step(val_loss)
    
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Val Loss: {val_loss:.4f}")
    print(f"Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
    
    # ベストモデル保存
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
        }, 'best_model_runpods.pth')
        print("✅ ベストモデル保存")

print("\n🎉 学習完了!")
print(f"Best Validation Loss: {best_val_loss:.4f}")

## 📈 学習曲線の可視化

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Progress')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(train_losses[-10:], label='Train Loss (Last 10)')
plt.plot(val_losses[-10:], label='Val Loss (Last 10)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Recent Progress')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## 🔍 予測サンプル確認

In [ ]:
# 予測実行
model.eval()
sample_batch = next(iter(val_loader))
data = sample_batch['data'].to(device)
target = sample_batch['target'].to(device)

with torch.no_grad():
    output = model(data)
    pred = output.argmax(dim=1)

# CPUに移動
data_cpu = data.cpu().numpy()
target_cpu = target.cpu().numpy()
pred_cpu = pred.cpu().numpy()

# 最初のサンプルの中央スライスを表示
idx = 0
slice_idx = data_cpu.shape[-1] // 2

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(data_cpu[idx, 0, :, :, slice_idx], cmap='gray')
plt.title('Input')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(target_cpu[idx, :, :, slice_idx], cmap='tab20')
plt.title('Ground Truth')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(pred_cpu[idx, :, :, slice_idx], cmap='tab20')
plt.title('Prediction')
plt.axis('off')

plt.suptitle(f'Sample Prediction (Slice {slice_idx})')
plt.tight_layout()
plt.show()

# Dice係数計算
intersection = ((pred_cpu == 1) & (target_cpu == 1)).sum()
union = (pred_cpu == 1).sum() + (target_cpu == 1).sum()
dice = 2.0 * intersection / (union + 1e-8)
print(f"\nSample Dice Coefficient: {dice:.4f}")

## 💾 モデル保存

In [ ]:
# 最終モデル保存
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'train_losses': train_losses,
    'val_losses': val_losses,
}, 'final_model_runpods.pth')

print("✅ モデル保存完了")
print("   - best_model_runpods.pth (ベストモデル)")
print("   - final_model_runpods.pth (最終モデル)")

## 🎯 完了

Runpods環境での学習が完了しました！

### 次のステップ:
1. 保存したモデルで推論実行
2. Kaggle提出用ファイル作成
3. ハイパーパラメータ調整